In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings, math
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# --- Load dataset ---
df = pd.read_csv("amazon.csv")
df.columns = df.columns.str.strip().str.lower()

In [2]:
# Keep relevant cols
df = df[['product_name','category','discounted_price','actual_price',
         'discount_percentage','rating','rating_count']]

# Clean numeric columns
for col in ['discounted_price','actual_price']:
    df[col] = df[col].astype(str).str.replace("₹","",regex=False).str.replace(",","",regex=False)
    df[col] = pd.to_numeric(df[col], errors='coerce')


In [9]:
df['discount_percentage'] = df['discount_percentage'].astype(str).str.replace("%","",regex=False)
df['discount_percentage'] = pd.to_numeric(df['discount_percentage'], errors='coerce')

In [10]:
df['rating'] = pd.to_numeric(df['rating'], errors='coerce')
df['rating_count'] = pd.to_numeric(df['rating_count'].astype(str).str.replace(",","",regex=False), errors='coerce')

# Drop rows without target
df = df.dropna(subset=['discounted_price'])

# --- Features & Target ---
X = df[['category','actual_price','discount_percentage','rating','rating_count']]
y = df['discounted_price']

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

# --- Preprocess: OneHot + Imputer + Scaler ---
cat_features = ['category']
num_features = ['actual_price','discount_percentage','rating','rating_count']

preprocess = ColumnTransformer([
    ("cat", Pipeline([
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(handle_unknown="ignore"))
    ]), cat_features),
    ("num", Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())
    ]), num_features)
])

models = {
    "LinearRegression": LinearRegression(),
    "RandomForest": RandomForestRegressor(n_estimators=200, random_state=42),
    "GradientBoosting": GradientBoostingRegressor(random_state=42)
}

In [12]:
results = {}
for name, model in models.items():
    pipe = Pipeline([("pre",preprocess),("model",model)])
    pipe.fit(X_train,y_train)
    pred = pipe.predict(X_test)
    mse = mean_squared_error(y_test,pred)
    rmse = math.sqrt(mse)
    mae = mean_absolute_error(y_test,pred)
    r2 = r2_score(y_test,pred)
    results[name] = (rmse,mae,r2)
    print(f"\n{name}:\n RMSE={rmse:.2f} | MAE={mae:.2f} | R²={r2:.2f}")

    # Scatter plot
    plt.figure(figsize=(5,5))
    plt.scatter(y_test,pred,alpha=0.5)
    plt.plot([y_test.min(), y_test.max()],[y_test.min(),y_test.max()],"r--")
    plt.xlabel("Actual Discounted Price")
    plt.ylabel("Predicted Discounted Price")
    plt.title(f"{name}: Predicted vs Actual")
    plt.show()

In [13]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# --- GUI State ---
state = {"pipe": None, "name": None}

# Dropdown for model
model_dd = widgets.Dropdown(
    options=list(models.keys()),
    value="RandomForest",
    description="Model:"
)

# Inputs
cat_in   = widgets.Text(value="Computers&Accessories", description="Category:")
price_in = widgets.FloatText(value=1000, description="Actual Price:")
disc_in  = widgets.FloatText(value=20, description="Discount %:")
rating_in= widgets.FloatText(value=4.0, description="Rating:")
count_in = widgets.IntText(value=1000, description="Rating Count:")

train_btn = widgets.Button(description="(Re)Train", button_style="primary")
pred_btn  = widgets.Button(description="Predict", button_style="success")
out       = widgets.Output(layout={'border': '1px solid #ddd', 'padding':'8px'})

# --- Train handler ---
def on_train_clicked(_):
    with out:
        clear_output(wait=True)
        name = model_dd.value
        model = models[name]
        pipe = Pipeline([("pre",preprocess),("model",model)])
        pipe.fit(X_train,y_train)
        state["pipe"], state["name"] = pipe, name
        print(f"✅ Trained {name}")

# --- Predict handler ---
def on_predict_clicked(_):
    with out:
        if state["pipe"] is None:
            print("Train a model first.")
            return
        data = pd.DataFrame([{
            "category": cat_in.value,
            "actual_price": price_in.value,
            "discount_percentage": disc_in.value,
            "rating": rating_in.value,
            "rating_count": count_in.value
        }])
        pred = state["pipe"].predict(data)[0]
        print(f"Model: {state['name']} | Predicted Discounted Price: ₹{pred:.2f}")

train_btn.on_click(on_train_clicked)
pred_btn.on_click(on_predict_clicked)

# --- Layout ---
ui = widgets.VBox([
    widgets.HBox([model_dd, train_btn]),
    cat_in, price_in, disc_in, rating_in, count_in,
    pred_btn, out
])

display(ui)
